# Turkish XTTS Fine-tuning Setup

**⚠️ IMPORTANT: Before running any cells below, do this:**
1. Go to `Runtime` → `Restart runtime` in Colab menu
2. Wait for restart to complete
3. Then run cells 2-8 in order

In [ ]:
!python --version

# ===== GPU check: L4 (Recommended) ====

In [ ]:
# ===== 0. GPU check: L4 =====
!nvidia-smi

In [ ]:
# ===== 1. Clone your repo =====
%cd /content
!git clone https://github.com/ElkhanAbbasov/aiswhis.git
%cd aiswhis

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# ===== 3. Install TTS in isolated environment =====
%cd /content/aiswhis

# Use pip's --force-reinstall and ignore dependency conflicts
# This works because we'll use what's already in Colab for most deps
!pip install --force-reinstall --no-deps -q "TTS==0.22.0"
!pip install --force-reinstall --no-deps -q "trainer==0.0.36"

# Now install TTS dependencies that are compatible with Colab
!pip install -q "coqpit>=0.0.16"
!pip install -q "transformers>=4.33.0"
!pip install -q "encodec>=0.1.1"
!pip install -q "pysbd>=0.3.4"
!pip install -q "gruut[de,es,fr]>=2.2.3"

print("Installation complete. Now verifying...")

In [ ]:
# ===== 3b. Verify installation and register safe globals =====
import sys
import torch
import numpy
print("Python:", sys.version)
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("numpy:", numpy.__version__)

# Test TTS import
try:
    from TTS.tts.configs.xtts_config import XttsConfig
    from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
    from TTS.config.shared_configs import BaseDatasetConfig
    print("✓ TTS imported successfully")
    
    # Register safe globals
    torch.serialization.add_safe_globals([
        XttsConfig, XttsAudioConfig, BaseDatasetConfig, XttsArgs
    ])
    print("✓ Safe globals registered")
except ImportError as e:
    print(f"❌ TTS import failed: {e}")
    print("Try restarting runtime and running cells again")

In [ ]:
# ===== 4. Verify dataset on Google Drive =====
import os

dataset_root = "/content/drive/MyDrive/xtts_tr_dataset"
wav_dir = os.path.join(dataset_root, "wavs")
meta_path = os.path.join(dataset_root, "metadata.txt")

wav_files = [f for f in os.listdir(wav_dir) if f.endswith(".wav")] if os.path.isdir(wav_dir) else []
print("Dataset root:", dataset_root)
print("Wavs dir exists:", os.path.isdir(wav_dir))
print("Number of wav files:", len(wav_files))
print("Metadata exists:", os.path.isfile(meta_path))
print("Metadata path:", meta_path)

In [ ]:
# ===== 5. Launch training =====
import os

os.environ["XTTS_TR_DATASET_ROOT"] = "/content/drive/MyDrive/xtts_tr_dataset"
os.environ["XTTS_TR_METADATA_FILE"] = "/content/drive/MyDrive/xtts_tr_dataset/metadata.txt"

%cd /content/aiswhis
!python train_gpt_xtts_tr.py